In [1]:
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
import dotenv
import time
dotenv.load_dotenv()
import os

llm = ChatOpenAI(model="gpt-4o", api_key=os.getenv('OPENAI_API_KEY'))

embeddings = OpenAIEmbeddings()

In [2]:
import os

os.environ["GOOGLE_CLOUD_PROJECT"] = 'clasificationfromdescription'
os.environ["GOOGLE_CLOUD_LOCATION"] = 'us-central1'
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = 'True'

In [5]:
from google import genai
from google.genai.types import HttpOptions

from backend.llms.llm_rag import LLMProtocol

class RagResponse():
    def __init__(self, text = None):
        self.text = text
model="gemini-1.0-flash"
class LLMGemini2(LLMProtocol):
    def __init__(self, config=None):
        self.client = genai.Client(http_options=HttpOptions(api_version="v1"))
 
    def process_request(self, message):
       
        response = self.client.models.generate_content(
            model=model,
            contents=message,
        )
        response_rag  = RagResponse()
        response_rag.text = response.text
        response_rag.raw_response = response
        return response_rag
    def log(self ):
        pass
backend_llm = LLMGemini2()
response = backend_llm.process_request("why is six afraid of seven ?")
# Example response:
# Okay, let's break down how AI works. It's a broad field, so I'll focus on the ...
#
# Here's a simplified overview:
# ...

ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'Publisher Model `projects/clasificationfromdescription/locations/us-central1/publishers/google/models/gemini-1` not found.', 'status': 'NOT_FOUND'}}

In [5]:
response.text

'This is a classic kid\'s joke!  The answer is:\n\nBecause **seven ate nine**. \n\nIt plays on the sound of "ate" and "eight".  😊 \n'

In [6]:
# Load my LLM 

import sys
import os
sys.path.append('../..')


from backend.llms.llm_rag import LLMRag
#from backend.llms.llm_gemini import LLMGemini


llmrag = LLMRag(backend=backend_llm)
response = llmrag.process_request('Ce e un subquery  ?')

config {'chunker_name': 'chunks', 'similarity_threshold': 0.2, 'top_k': 5, 'no_internal_source_prompt': 'Your only answer should be exactly like this: Nu am gasit informatii in curs despre intrebarea ta', 'internal_source_rules': 'Esti un asistent care raspunde in limba Romana la intrebarea din  "ORIGINAL USER PROMPT" folosind context din "CONTEXT FROM INTERNAL SOURCES". Raspunde folosind numai informatii din context. Raspunsul tau trebuie sa fie complet si sa contina cat mai mult din contextul intrebarii. Vei raspunde \'Folosind informatiile din curs, \' si apoi vei raspunde'} 
 -------------------------------
No embeddings db directory found!
Loading documents..
Splitting by chunks

loading documents..
Current directory: /data/ai/dirigu/teaching_helper


100%|██████████| 14/14 [00:03<00:00,  3.82it/s]


TypeError: 'list' object is not callable

In [6]:
import pandas as pd
intrebari_df = pd.read_csv("ragas_results_G1.5.csv")[0:10]

In [ ]:
intrebari_df 

In [8]:
# sample_queries = [intrebari_dict[x]['query'] for x in intrebari_df]
# expected_responses = [intrebari_dict[x]['expected_response'] for x in intrebari_dict]
sample_queries = intrebari_df["user_input"].values

expected_responses = intrebari_df["reference"].values
expected_responses = [x.removeprefix("Folosind informatiile din curs") for x in expected_responses]

In [ ]:
expected_responses

In [ ]:
dataset = []
i=0
for query,reference in zip(sample_queries,expected_responses):
    print(f'question{i}')
    i+=1
    full_response = llmrag.process_request(query)
    time.sleep(10)
    relevant_docs = full_response.context
    response = full_response.text.removeprefix("Folosind informatiile din curs, ")
    dataset.append(
        {
            "user_input":query,
            "retrieved_contexts":relevant_docs,
            "response":response,
            "reference":reference
        }
    )

In [ ]:
dataset

In [12]:
from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

In [ ]:
evaluation_dataset.to_pandas()

In [ ]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper


evaluator_llm = LangchainLLMWrapper(llm)
from ragas.metrics import ContextPrecision,  LLMContextRecall, AnswerRelevancy,  Faithfulness,  BleuScore, RougeScore, FactualCorrectness
metrics = [
        ContextPrecision(),
        LLMContextRecall(),
        AnswerRelevancy(),
        Faithfulness(),
        BleuScore(),
        RougeScore(),
        FactualCorrectness()]

result = evaluate(
    dataset=evaluation_dataset,
    metrics=metrics,
    llm=evaluator_llm)
result

In [15]:
import os
os.environ["RAGAS_APP_TOKEN"] = "apt.4418-2f131daa8aa7-086e-8dd7-0123d22b-08626"

In [ ]:
print(f"ragas_results_{model}_{llmrag.config['chunker_name']}.csv")
result.to_pandas().to_csv(f"ragas_results_{model}_{llmrag.config['chunker_name']}.csv")

In [17]:
#result.upload()